In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from sentence_transformers import SentenceTransformer
from torch.utils.data import DataLoader, TensorDataset
from pytorch_lightning.callbacks import Callback
import pandas as pd
import numpy as np
from pytorch_lightning.callbacks import ModelCheckpoint

D:\Anaconda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060 Laptop GPU'

In [17]:
users = pd.read_csv('processed_dataset/MovieLens-1M/users/users_movielens.csv')
movies = pd.read_csv('processed_dataset/MovieLens-1M/movies/movies_movielens_modified.csv')
# movies = pd.read_csv('processed_dataset/MovieLens-1M/movies/movies_movielens.csv')

full_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/ml_1m_full_movielens.csv')
train_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/ml_1m_train_movielens.csv')
val_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/ml_1m_val_movielens.csv')
test_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/ml_1m_test_movielens.csv')

In [5]:
# Combine user features into a single string for each user
users['user_features'] = 'occupation: ' + users['occupation'] + ' [SEP] age: ' + users['age'].astype(str) + ' [SEP] gender: ' + users['gender'].astype(str)

# Combine movie features into a single string for each movie
movies['movie_features'] = 'title: ' + movies['title'] + ' [SEP] genres: ' + movies['genres']

In [18]:
# Combine user features into a single string for each user
# users['user_features'] = '[USER_PROFILE] occupation: ' + users['occupation'] + ' [SEP] age: ' + users['age'].astype(str) + ' [SEP] gender: ' + users['gender'].astype(str)
users['user_features'] = '[USER_PROFILE] occupation: ' + users['occupation'] + ' [SEP] gender: ' + users['gender'].astype(str)

# Combine movie features into a single string for each movie
# movies['movie_features'] = '[MOVIE_DETAIL] title: ' + movies['title'] + ' genres: ' + movies['genres']
movies['movie_features'] = '[MOVIE_DETAIL] genres: ' + movies['genres']

In [19]:
users

,user_id,gender,age,occupation,zip_code,user_features
0,1,Female,Under 18,K-12 student,48067,[USER_PROFILE] occupation: K-12 student [SEP] ...
1,2,Male,56+,self-employed,70072,[USER_PROFILE] occupation: self-employed [SEP]...
2,3,Male,25-34,scientist,55117,[USER_PROFILE] occupation: scientist [SEP] gen...
3,4,Male,45-49,executive/managerial,02460,[USER_PROFILE] occupation: executive/manageria...
4,5,Male,25-34,writer,55455,[USER_PROFILE] occupation: writer [SEP] gender...
...,...,...,...,...,...,...
6035,6036,Female,25-34,scientist,32603,[USER_PROFILE] occupation: scientist [SEP] gen...
6036,6037,Female,45-49,academic/educator,76006,[USER_PROFILE] occupation: academic/educator [...
6037,6038,Female,56+,academic/educator,14706,[USER_PROFILE] occupation: academic/educator [...
6038,6039,Female,45-49,other or not specified,01060,[USER_PROFILE] occupation: other or not specif...


In [20]:
movies

,item_id,title,genres,year,movie_features
0,1,Toy Story,"Animation, Children's, Comedy",1995,"[MOVIE_DETAIL] genres: Animation, Children's, ..."
1,2,Jumanji,"Adventure, Children's, Fantasy",1995,"[MOVIE_DETAIL] genres: Adventure, Children's, ..."
2,3,Grumpier Old Men,"Comedy, Romance",1995,"[MOVIE_DETAIL] genres: Comedy, Romance"
3,4,Waiting to Exhale,"Comedy, Drama",1995,"[MOVIE_DETAIL] genres: Comedy, Drama"
4,5,Father of the Bride Part II,Comedy,1995,[MOVIE_DETAIL] genres: Comedy
...,...,...,...,...,...
3878,3948,Meet the Parents,Comedy,2000,[MOVIE_DETAIL] genres: Comedy
3879,3949,Requiem for a Dream,Drama,2000,[MOVIE_DETAIL] genres: Drama
3880,3950,Tigerland,Drama,2000,[MOVIE_DETAIL] genres: Drama
3881,3951,Two Family House,Drama,2000,[MOVIE_DETAIL] genres: Drama


In [21]:
# Create a dictionary for fast lookup
user_features_dict = users.set_index('user_id')['user_features'].to_dict()
movie_features_dict = movies.set_index('item_id')['movie_features'].to_dict()

# # Create lists of user and item texts
user_texts = [user_features_dict[userId] for userId in full_ratings['user_id'].unique()]
item_texts = [movie_features_dict[movieId] for movieId in full_ratings['item_id'].unique()]

# Create a mapping from userId and movieId to indices
user_id_to_idx = {userId: idx for idx, userId in enumerate(full_ratings['user_id'].unique())}
movie_id_to_idx = {movieId: idx for idx, movieId in enumerate(full_ratings['item_id'].unique())}

# Map userId and movieId in ratings_df to indices
train_ratings['user_idx'] = train_ratings['user_id'].map(user_id_to_idx)
train_ratings['movie_idx'] = train_ratings['item_id'].map(movie_id_to_idx)

# Map userId and movieId in ratings_val to indices
val_ratings['user_idx'] = val_ratings['user_id'].map(user_id_to_idx)
val_ratings['movie_idx'] = val_ratings['item_id'].map(movie_id_to_idx)

# Map userId and movieId in ratings_val to indices
test_ratings['user_idx'] = test_ratings['user_id'].map(user_id_to_idx)
test_ratings['movie_idx'] = test_ratings['item_id'].map(movie_id_to_idx)

# Extract user indices, item indices, and ratings
train_user_indices = torch.LongTensor(train_ratings['user_idx'].values).to(device)
train_item_indices = torch.LongTensor(train_ratings['movie_idx'].values).to(device)
train_labels = torch.FloatTensor(train_ratings['rating'].values).to(device)

# Extract user indices, item indices, and ratings for validation
val_user_indices = torch.LongTensor(val_ratings['user_idx'].values).to(device)
val_item_indices = torch.LongTensor(val_ratings['movie_idx'].values).to(device)
val_labels = torch.FloatTensor(val_ratings['rating'].values).to(device)

test_user_indices = torch.LongTensor(test_ratings['user_idx'].values).to(device)
test_item_indices = torch.LongTensor(test_ratings['movie_idx'].values).to(device)
test_labels = torch.FloatTensor(test_ratings['rating'].values).to(device)

In [26]:
train_ratings

,user_id,item_id,rating,timestamp,user_idx,movie_idx
0,1,3186,4,978300019,6033,1773
1,1,1022,5,978300055,6033,135
2,1,1721,4,978300055,6033,203
3,1,1270,5,978300055,6033,103
4,1,2340,3,978300103,6033,1649
...,...,...,...,...,...,...
797753,6040,2612,5,960971797,0,1909
797754,6040,2303,5,960971857,0,292
797755,6040,3504,4,960971857,0,224
797756,6040,1449,3,960971857,0,1396


In [31]:
train_item_indices[0].item()

1773

In [32]:
item_texts[train_item_indices[0].item()]

'[MOVIE_DETAIL] genres: Drama'

In [33]:
train_user_indices[0].item()

6033

In [34]:
user_texts[train_user_indices[0].item()]

'[USER_PROFILE] occupation: K-12 student [SEP] gender: Female'

In [35]:
train_labels[0].item()

4.0

In [22]:
# Create DataLoader for training data
train_dataset = TensorDataset(train_user_indices, train_item_indices, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)

# Create DataLoader for training data
val_dataset = TensorDataset(val_user_indices, val_item_indices, val_labels)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True, drop_last=True)

# Create DataLoader for training data
test_dataset = TensorDataset(test_user_indices, test_item_indices, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True, drop_last=True)

In [23]:
class TwoTowerModel(pl.LightningModule):
    def __init__(self, user_model_name, item_model_name, embedding_size=384):
        super(TwoTowerModel, self).__init__()
        self.user_model = SentenceTransformer(user_model_name)
        self.item_model = SentenceTransformer(item_model_name)

        self.user_fc = nn.Linear(embedding_size, embedding_size)
        self.item_fc = nn.Linear(embedding_size, embedding_size)

        self.criterion = nn.MSELoss()
        self.epoch_losses = {'train_loss': [], 'val_loss': []}

    def forward(self, user_text, item_text):
        user_embedding = self.user_model.encode(user_text, convert_to_tensor=True).to(device)
        item_embedding = self.item_model.encode(item_text, convert_to_tensor=True).to(device)

        user_output = self.user_fc(user_embedding)
        item_output = self.item_fc(item_embedding)

        dot_product = torch.matmul(user_output.unsqueeze(1), item_output.unsqueeze(2)).squeeze()

        dot_product = 4 * torch.sigmoid(dot_product) + 1

        return dot_product

    def training_step(self, batch, batch_idx):
        users, items, ratings = batch

        users = [user_texts[i] for i in users.tolist()]
        items = [item_texts[i] for i in items.tolist()]

        preds = self(users, items)

        loss = self.criterion(preds, ratings)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        users, items, ratings = batch

        users = [user_texts[i] for i in users.tolist()]
        items = [item_texts[i] for i in items.tolist()]

        preds = self(users, items)

        loss = self.criterion(preds, ratings)
        self.log('val_loss', loss)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=1e-5)

class PrintLossesCallback(Callback):
    def on_train_epoch_end(self, trainer, pl_module):
        train_loss = trainer.callback_metrics.get('train_loss')
        if train_loss is not None:
            pl_module.epoch_losses['train_loss'].append(train_loss.item())
            print(f"Epoch {trainer.current_epoch + 1}: Train Loss: {train_loss.item()}")

    def on_validation_epoch_end(self, trainer, pl_module):
        val_loss = trainer.callback_metrics.get('val_loss')
        if val_loss is not None:
            pl_module.epoch_losses['val_loss'].append(val_loss.item())
            print(f"Epoch {trainer.current_epoch + 1}: Val Loss: {val_loss.item()}")

In [38]:
# model = TwoTowerModel(user_model_name='all-MiniLM-L6-v2', item_model_name='all-MiniLM-L6-v2')
# model = TwoTowerModel(user_model_name='paraphrase-MiniLM-L6-v2', item_model_name='paraphrase-MiniLM-L6-v2')
model = TwoTowerModel(user_model_name='paraphrase-MiniLM-L12-v2', item_model_name='paraphrase-MiniLM-L12-v2')

# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Metric to monitor
    dirpath='checkpoints/',  # Directory to save the checkpoints
    filename='no-history-best-checkpoint',  # Filename for the best model
    save_top_k=1,  # Save only the top 1 model
    mode='min'  # Mode to save the best model (min for validation loss)
)

trainer = pl.Trainer(max_epochs=5, log_every_n_steps=1, callbacks=[PrintLossesCallback()], enable_progress_bar=True)
trainer.fit(model, train_dataloader, val_dataloader)

# Print losses after training completes
print("Epoch losses:")
for epoch in range(trainer.max_epochs):
    train_loss = model.epoch_losses['train_loss'][epoch] if epoch < len(model.epoch_losses['train_loss']) else 'N/A'
    val_loss = model.epoch_losses['val_loss'][epoch] if epoch < len(model.epoch_losses['val_loss']) else 'N/A'
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss}, Val Loss: {val_loss}")

D:\Anaconda\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type                | Params | Mode 
-----------------------------------------------------------
0 | user_model | SentenceTransformer | 33.4 M | train
1 | item_model | SentenceTransformer | 33.4 M | train
2 | user_fc    | Linear              | 147 K  | train
3 | item_fc    | Linear              | 147 K  | train
4 | criterion  | MSELoss             | 0      | train
-----------------------------------------------------------
67.0 M    Trainable params
0         Non-trainable params
67.0 M    Total params
268.063   Total estimated model params siz

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

D:\Anaconda\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:475: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
D:\Anaconda\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  5.50it/s]Epoch 1: Val Loss: 1.7421693801879883
                                                                           

D:\Anaconda\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 12464/12464 [17:39<00:00, 11.76it/s, v_num=33]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████| 1557/1557 [02:40<00:00,  9.68it/s]Epoch 1: Val Loss: 1.2011321783065796

Epoch 1: 100%|██████████| 12464/12464 [23:40<00:00,  8.78it/s, v_num=33]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████| 1557/1557 [03:37<00:00,  7.16it/s]Epoch 2: Val Loss: 1.1996442079544067

Epoch 2: 100%|██████████| 12464/12464 [18:50<00:00, 11.02it/s, v_num=33]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████| 1557/1557 [02:48<00:00,  9.23it/s]Epoch 3: Val Loss: 1.216110348701477

Epoch 3: 100%|██████████| 12464/12464 [22:50<00:00,  9.10it/s, v_num=33]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████| 1557/1557 [02:41<00:00,  9.64it/s]Epoch 4: Val Loss: 1.2009116411209106

Epoch 4: 100%|██████████| 12464/12464 [24:49<00:00,  8.37it/s, v_

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 12464/12464 [27:44<00:00,  7.49it/s, v_num=33]
Epoch losses:
Epoch 1: Train Loss: 1.4573062658309937, Val Loss: 1.7421693801879883
Epoch 2: Train Loss: 1.0970215797424316, Val Loss: 1.2011321783065796
Epoch 3: Train Loss: 0.968858003616333, Val Loss: 1.1996442079544067
Epoch 4: Train Loss: 1.253294825553894, Val Loss: 1.216110348701477
Epoch 5: Train Loss: 1.4069586992263794, Val Loss: 1.2009116411209106


In [30]:
class ImprovedTwoTowerModel(pl.LightningModule):
    def __init__(self, user_model_name, item_model_name, embedding_size=384, hidden_units=64, dropout_rate=0.5):
        super(ImprovedTwoTowerModel, self).__init__()
        self.user_model = SentenceTransformer(user_model_name)
        self.item_model = SentenceTransformer(item_model_name)

        self.user_fc = nn.Sequential(
            nn.Linear(embedding_size, hidden_units),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_units, embedding_size)
        )
        self.item_fc = nn.Sequential(
            nn.Linear(embedding_size, hidden_units),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_units, embedding_size)
        )

        self.criterion = nn.MSELoss()
        self.epoch_losses = {'train_loss': [], 'val_loss': []}

    def forward(self, user_text, item_text):
        user_embedding = self.user_model.encode(user_text, convert_to_tensor=True).to(device)
        item_embedding = self.item_model.encode(item_text, convert_to_tensor=True).to(device)

        user_output = self.user_fc(user_embedding)
        item_output = self.item_fc(item_embedding)

        dot_product = torch.matmul(user_output.unsqueeze(1), item_output.unsqueeze(2)).squeeze()
        dot_product = 4 * torch.sigmoid(dot_product) + 1

        return dot_product

    def training_step(self, batch, batch_idx):
        users, items, ratings = batch

        users = [user_texts[i] for i in users.tolist()]
        items = [item_texts[i] for i in items.tolist()]

        preds = self(users, items)

        loss = self.criterion(preds, ratings)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        users, items, ratings = batch

        users = [user_texts[i] for i in users.tolist()]
        items = [item_texts[i] for i in items.tolist()]

        preds = self(users, items)

        loss = self.criterion(preds, ratings)
        self.log('val_loss', loss)
        return loss

    def full_predict(self, user_ids, item_ids):
        users = [user_texts[i] for i in user_ids.tolist()]
        items = [item_texts[i] for i in item_ids.tolist()]
        user_embedding = self.user_model.encode(users, convert_to_tensor=True).to(self.device)
        item_embeddings = torch.stack([self.item_model.encode(item_text, convert_to_tensor=True) for item_text in items]).to(self.device)

        user_output = self.user_fc(user_embedding)
        item_output = self.item_fc(item_embeddings)

        dot_product = torch.matmul(user_output, item_output.T)
        dot_product = 4 * torch.sigmoid(dot_product) + 1

        return dot_product

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=1e-5)


class PrintLossesCallback(Callback):
    def on_train_epoch_end(self, trainer, pl_module):
        train_loss = trainer.callback_metrics.get('train_loss')
        if train_loss is not None:
            pl_module.epoch_losses['train_loss'].append(train_loss.item())
            print(f"Epoch {trainer.current_epoch + 1}: Train Loss: {train_loss.item()}")

    def on_validation_epoch_end(self, trainer, pl_module):
        val_loss = trainer.callback_metrics.get('val_loss')
        if val_loss is not None:
            pl_module.epoch_losses['val_loss'].append(val_loss.item())
            print(f"Epoch {trainer.current_epoch + 1}: Val Loss: {val_loss.item()}")

In [ ]:
# model = TwoTowerModel(user_model_name='all-MiniLM-L6-v2', item_model_name='all-MiniLM-L6-v2')
model = ImprovedTwoTowerModel(user_model_name='paraphrase-MiniLM-L6-v2', item_model_name='paraphrase-MiniLM-L6-v2')

# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Metric to monitor
    dirpath='checkpoints/',  # Directory to save the checkpoints
    filename='no-history-best-checkpoint',  # Filename for the best model
    save_top_k=1,  # Save only the top 1 model
    mode='min'  # Mode to save the best model (min for validation loss)
)

trainer = pl.Trainer(max_epochs=5, log_every_n_steps=1, callbacks=[PrintLossesCallback()], enable_progress_bar=True)
trainer.fit(model, train_dataloader, val_dataloader)

# Print losses after training completes
print("Epoch losses:")
for epoch in range(trainer.max_epochs):
    train_loss = model.epoch_losses['train_loss'][epoch] if epoch < len(model.epoch_losses['train_loss']) else 'N/A'
    val_loss = model.epoch_losses['val_loss'][epoch] if epoch < len(model.epoch_losses['val_loss']) else 'N/A'
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss}, Val Loss: {val_loss}")

In [ ]:
model.epoch_losses

# Evaluation

In [24]:
# Assuming the training part has been done already, load the best model checkpoint
# # best_model_path = './lightning_logs/paraphrase-MiniLM-L6-v2/not-binarized/no-history_5-epochs_lr-1e-5/checkpoints/epoch=4-step=93765.ckpt'  # Path where the best model is saved
# best_model_path = './lightning_logs/paraphrase-MiniLM-L6-v2/not-binarized/no-history_5-epochs_lr-1e-5_(new format - only genre for movies)/checkpoints/epoch=4-step=62320.ckpt'  # Path where the best model is saved
best_model_path = './lightning_logs/paraphrase-MiniLM-L12-v2/not-binarized/no-history_5-epochs_lr-1e-5_(new format - only genre for movies)/checkpoints/epoch=4-step=62320.ckpt'  # Path where the best model is saved

# best_model = TwoTowerModel.load_from_checkpoint(best_model_path, user_model_name='paraphrase-MiniLM-L6-v2', item_model_name='paraphrase-MiniLM-L6-v2').to(device)
best_model = TwoTowerModel.load_from_checkpoint(best_model_path, user_model_name='paraphrase-MiniLM-L12-v2', item_model_name='paraphrase-MiniLM-L12-v2').to(device)

# paraphrase-MiniLM-L12-v2

D:\Anaconda\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Calculations

In [36]:
def get_top_n_items_without_history_unseen_items(model, userId, n):
    # Ensure the model is in evaluation mode
    model.eval()

    # Get the user text for the given userId
    user_text = user_features_dict[userId]

    # Encode the user text
    user_embedding = model.user_model.encode(user_text, convert_to_tensor=True).to(device)

    # Compute the scores (dot product between user embedding and each item embedding)
    user_output = model.user_fc(user_embedding).to(device)
    item_output = model.item_fc(full_items_embeddings).to(device)
    dot_product = torch.matmul(user_output, item_output.t()).squeeze()

    # Get items the user has seen in the training and validation data
    seen_items_train = train_ratings[train_ratings['user_id'] == userId]['item_id'].values
    seen_items_val = val_ratings[val_ratings['user_id'] == userId]['item_id'].values
    seen_items = set(np.concatenate((seen_items_train, seen_items_val)))
    print(seen_items)
    # Get the top n + len(seen_items) item indices and their scores
    top_n_scores, top_n_indices = torch.topk(dot_product, n + len(seen_items))

    # Map indices back to item IDs
    top_n_item_ids = [list(movie_id_to_idx.keys())[list(movie_id_to_idx.values()).index(idx.item())] for idx in top_n_indices]
    # print(top_n_item_ids[:5])
    # Filter out seen items
    unseen_top_n_item_ids = [item for item in top_n_item_ids if item not in seen_items]
    # print(unseen_top_n_item_ids[:n])
    return unseen_top_n_item_ids[:n]

In [26]:
# Assuming full_items_embeddings is already defined
full_items_embeddings = torch.stack([best_model.item_model.encode(item_text, convert_to_tensor=True) for item_text in item_texts]).to(device)

## Type 0

In [37]:
def dcg(scores, k):
    scores = np.asfarray(scores)[:k]
    return np.sum(scores / np.log2(np.arange(2, scores.size + 2)))

def ndcg_at_k(labels, k):
    ideal_labels = sorted(labels, reverse=True)
    return dcg(labels, k) / dcg(ideal_labels, k)

def recall_at_k(labels, relevant_count, k):
    return np.sum(labels[:k]) / relevant_count

def mrr_at_k(labels, k):
    for i, label in enumerate(labels[:k]):
        if label == 1:
            return 1 / (i + 1)
    return 0

def evaluate_user_cf_model(model, test_data, train_data, val_data, all_items, k):
    ndcg_scores = []
    recall_scores = []
    mrr_scores = []

    # Get unique users
    unique_users = test_data['user_id'].unique()

    for user in unique_users:
        # Get the top N items for the user, filtering out seen items
        recommended_items = get_top_n_items_without_history_unseen_items(model, user, k)

        user_test_data = test_data[test_data['user_id'] == user]
        test_items = user_test_data['item_id'].values

        y_score = [1 if item in test_items else 0 for item in recommended_items]

        ndcg = ndcg_at_k(y_score, k)
        recall = recall_at_k(y_score, len(test_items), k)
        mrr = mrr_at_k(y_score, k)

        ndcg_scores.append(ndcg)
        recall_scores.append(recall)
        mrr_scores.append(mrr)

    avg_ndcg = np.nanmean(ndcg_scores)
    avg_recall = np.nanmean(recall_scores)
    avg_mrr = np.nanmean(mrr_scores)

    return {
        'NDCG@{}'.format(k): avg_ndcg,
        'Recall@{}'.format(k): avg_recall,
        'MRR@{}'.format(k): avg_mrr,
    }

all_items = movies['item_id'].unique()
# Evaluate the model
eval_result = evaluate_user_cf_model(best_model, test_ratings, train_ratings, val_ratings, all_items, k=5)
print(eval_result)
eval_result = evaluate_user_cf_model(best_model, test_ratings, train_ratings, val_ratings, all_items, k=10)
print(eval_result)

{1, 260, 2692, 1028, 1287, 1029, 1545, 1035, 527, 2321, 914, 531, 661, 150, 919, 3105, 2340, 1193, 1961, 938, 1836, 1962, 3114, 1197, 1207, 1721, 1097, 2762, 588, 3408, 720, 594, 595, 1246, 2398, 608, 2018, 2918, 2791, 745, 2028, 2797, 3186, 2804, 1270, 1022, 2687}
{1537, 515, 2571, 1552, 2067, 21, 3095, 3105, 3107, 3108, 1084, 1597, 1090, 3654, 1096, 1610, 3147, 589, 590, 1103, 593, 3678, 1124, 110, 3699, 647, 648, 2194, 3735, 2717, 163, 1188, 165, 2728, 1193, 1196, 1198, 1207, 3255, 3256, 1210, 2236, 1213, 1217, 1225, 1244, 1245, 2268, 1247, 1246, 3809, 1253, 2278, 1259, 235, 1265, 1784, 1792, 3334, 2312, 265, 1801, 780, 1293, 2321, 2852, 2858, 1834, 2353, 3893, 2359, 318, 2881, 1357, 1873, 3418, 1370, 2396, 349, 1372, 356, 2916, 1385, 368, 3451, 380, 2427, 2943, 1408, 902, 3468, 3471, 920, 1945, 1953, 1954, 1955, 1442, 1957, 1962, 1968, 2490, 2501, 457, 459, 3030, 982, 2006, 3035, 480, 2028, 498, 1527, 3578, 3068, 3071}
{260, 648, 1291, 653, 1431, 1304, 1049, 3619, 2470, 552, 1961, 

C:\Users\Hooman\AppData\Local\Temp\ipykernel_28916\183069761.py:7: RuntimeWarning: invalid value encountered in scalar divide
  return dcg(labels, k) / dcg(ideal_labels, k)


{6, 648, 2571, 1573, 3753, 1196, 1580, 3256, 442, 1221, 457, 1610, 1997, 589, 3793, 3418, 474, 733, 861, 480, 2916, 2028, 110, 1270, 377, 3578, 380}
{2571, 524, 14, 527, 16, 17, 24, 25, 538, 3105, 3107, 1059, 1573, 36, 39, 2600, 555, 1580, 562, 1589, 58, 73, 3147, 3148, 589, 1621, 608, 1120, 3173, 1639, 110, 111, 3186, 1653, 1660, 2686, 2688, 2692, 1673, 650, 2699, 1678, 2702, 1682, 150, 151, 2712, 1693, 161, 163, 1701, 1704, 3246, 1711, 3250, 3252, 3256, 1721, 1210, 3259, 3260, 1213, 3265, 1730, 3267, 1735, 2268, 733, 2278, 230, 1265, 2291, 2297, 1277, 253, 265, 266, 269, 2320, 1810, 2324, 2329, 282, 288, 2336, 296, 2858, 1836, 1840, 3386, 1357, 337, 345, 3418, 2396, 349, 2908, 3425, 2916, 1393, 377, 2427, 1916, 2429, 1411, 393, 2442, 908, 3481, 2490, 1466, 3006, 1476, 454, 3528, 1488, 465, 2006, 476, 480, 2023, 2028, 2541, 506, 508}
{1, 2571, 527, 1552, 25, 1060, 3623, 2599, 3114, 47, 1584, 50, 1089, 3147, 3148, 590, 593, 1617, 597, 3160, 608, 1639, 3178, 2166, 1148, 2692, 3717, 1669

KeyboardInterrupt: 

In [ ]:
{'NDCG@5': 0.6738031382968465}

## Type 2

In [28]:
def evaluate_user_cf_model(model, test_data, train_data, val_data, all_items, k):
    ndcg_scores = []

    # Get unique users
    unique_users = test_data['user_id'].unique()

    for user in unique_users:
        # Get the top N items for the user, filtering out seen items
        recommended_items = get_top_n_items_without_history_unseen_items(model, user, k)

        user_test_data = test_data[test_data['user_id'] == user]
        test_items = user_test_data['item_id'].values
        # print(user)
        y_score = [
            user_test_data[user_test_data['item_id'] == item]['rating'].values[0] if item in test_items else 2.5
            for item in recommended_items
        ]
        ndcg = ndcg_at_k(y_score, k)
        ndcg_scores.append(ndcg)

    avg_ndcg = np.nanmean(ndcg_scores)

    return {
        'NDCG@{}'.format(k): avg_ndcg
    }

all_items = movies['item_id'].unique()
# Evaluate the model
# eval_result = evaluate_user_cf_model(best_model, test_ratings, train_ratings, val_ratings, all_items, k=5)
# print(eval_result)
eval_result = evaluate_user_cf_model(best_model, test_ratings, train_ratings, val_ratings, all_items, k=10)
print(eval_result)

{'NDCG@10': 0.9993909452845933}


## Type 3

In [29]:
def evaluate_user_cf_model(model, test_data, train_data, val_data, all_items, k):
    ndcg_scores = []

    # Get unique users
    unique_users = test_data['user_id'].unique()

    for user in unique_users:
        # Get the top N items for the user, filtering out seen items
        recommended_items = get_top_n_items_without_history_unseen_items(model, user, k)

        user_test_data = test_data[test_data['user_id'] == user]
        test_items = user_test_data['item_id'].values

        y_score = [
            user_test_data[user_test_data['item_id'] == item]['rating'].values[0] if item in test_items else 0
            for item in recommended_items
        ]
        ndcg = ndcg_at_k(y_score, k)
        ndcg_scores.append(ndcg)

    avg_ndcg = np.nanmean(ndcg_scores)

    return {
        'NDCG@{}'.format(k): avg_ndcg
    }

all_items = movies['item_id'].unique()

# Evaluate the model
# eval_result = evaluate_user_cf_model(best_model, test_ratings, train_ratings, val_ratings, all_items, k=5)
# print(eval_result)
eval_result = evaluate_user_cf_model(best_model, test_ratings, train_ratings, val_ratings, all_items, k=10)
print(eval_result)

C:\Users\Hooman\AppData\Local\Temp\ipykernel_28916\183069761.py:7: RuntimeWarning: invalid value encountered in scalar divide
  return dcg(labels, k) / dcg(ideal_labels, k)


{'NDCG@10': 0.6995491470393821}


## Type 1

In [ ]:
def evaluate_user_cf_model(model, test_data, train_data, val_data, all_items, k):
    ndcg_scores = []

    # Get unique users
    unique_users = test_data['user_id'].unique()

    for user in unique_users:
        # Get the top N items for the user, filtering out seen items
        recommended_items = get_top_n_items_without_history_unseen_items(model, user, k)

        user_test_data = test_data[test_data['user_id'] == user]
        test_items = user_test_data['item_id'].values
        print(user)
        # y_score = [
        #     user_test_data[user_test_data['item_id'] == item]['rating'].values[0] if item in test_items else 0
        #     for item in recommended_items
        # ]
        y_score = [
            1 if (item in test_items and user_test_data[user_test_data['item_id'] == item]['label'].values[0] == 1) else 0
            for item in recommended_items
        ]
        # y_score = [
        #     1 if (item in test_items and user_test_data[user_test_data['item'] == item]['label'].values[0] == 1) else 0
        #     for item in recommended_items
        # ]

        ndcg = ndcg_at_k(y_score, k)
        ndcg_scores.append(ndcg)

    avg_ndcg = np.nanmean(ndcg_scores)

    return {
        'NDCG@{}'.format(k): avg_ndcg
    }

all_items = movies['item_id'].unique()
# Evaluate the model
eval_result = evaluate_user_cf_model(best_model, test_ratings, train_ratings, val_ratings, all_items, k=5)
print(eval_result)
eval_result = evaluate_user_cf_model(best_model, test_ratings, train_ratings, val_ratings, all_items, k=5)
print(eval_result)